In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sqlalchemy
import pyodbc
#%matplotlib inline
plt.rcParams['figure.figsize']=[10,5]
pd.set_option('display.max_colwidth', -1)
import re
from collections import Counter
from numpy import log
import warnings 
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
import os as os
from sqlalchemy import create_engine
engine=create_engine('mssql+pyodbc://ClarityProd')
engine_DW =create_engine('mssql+pyodbc://prodedw')
import os.path, time
from pandas.plotting import scatter_matrix
from datetime import datetime, date, timedelta
import urllib

svm_sql ='''
SELECT Date
      ,fit
  FROM [BI_Reporting].[OLYMPUS\\tflaherty].[SVM_COVID_FORECAST] 
'''
census_total_pred =pd.read_sql_query(svm_sql, engine)
census_total_pred['Date'] =pd.to_datetime(census_total_pred['Date'])


census_sql ='''
with valid_census as 
(SELECT  
      *
  FROM [BI_Reporting].[dbo].[Hospital_census_trending]
  where COST_CENTER_NAME not in ('ADULT CRISIS UNIT','CARDIAC CATH LAB')
   and DEPARTMENT_NAME <>'NMC 6UNS'
   and isnull(bed_block,'N') <>'Y'
   AND Date_of_Census>=dateadd(mm, -2, getdate()) )

select TotalAvailable.censusdate, 
       totalavailable.total,
	   TOTALICU.totalICU,
	   Inpatient_CNT, 
	   ICU_CNT, 
	   COVID_CNT, 
	   covid_icu.COVID_ICU_CNT,
	   covid_None_icu.COVID_None_ICU_CNT from 
 (SELECT  
      count(BED_ID) total
     
      ,convert(date,[Date_of_Census]) censusdate
      
  FROM valid_census
  Group by convert(date,Date_of_census)) TotalAvailable
  left join
  (SELECT  
      count(BED_ID) totalICU
     
      ,convert(date,[Date_of_Census]) censusdate
      
  FROM valid_census
  WHERE unit_type ='ICU'
  Group by convert(date,Date_of_census)) totalICU
  ON TotalAvailable.censusdate=totalICU.censusdate
  left join 
 (
  SELECT  
      count(BED_ID) Inpatient_CNT
     
      ,convert(date,[Date_of_Census]) censusdate
      
  FROM valid_census
  WHERE mrn is not null and unit_type <>'ICU'
  Group by convert(date,Date_of_census)) Inpatient
  on Inpatient.censusdate=TotalAvailable.censusdate
 left  join
  (
SELECT  
      count(BED_ID) ICU_CNT
      
      ,convert(date,[Date_of_Census]) censusdate
      
  FROM valid_census
  WHERE mrn is not null
  and unit_type ='ICU'
  and Positive_COVID_This_Census_Date <>1
  Group by convert(date,Date_of_census)) ICU on ICU.censusdate=Inpatient.censusdate
  left JOIN (
  SELECT  
      count(BED_ID) COVID_CNT
     
      ,convert(date,[Date_of_Census]) censusdate
      
  FROM valid_census
  WHERE mrn is not null and Positive_COVID_This_Census_Date =1
  Group by convert(date,Date_of_census)
  ) COVID ON Inpatient.censusdate =COVID.censusdate
  left JOIN (
  SELECT  
      count(BED_ID) COVID_ICU_CNT
     
      ,convert(date,[Date_of_Census]) censusdate
      
  FROM valid_census
  WHERE mrn is not null and Positive_COVID_This_Census_Date =1
  AND unit_type ='ICU'
  Group by convert(date,Date_of_census)) covid_icu 
  on TotalAvailable.censusdate=covid_icu.censusdate
  left JOIN (
  SELECT  
      count(BED_ID) COVID_None_ICU_CNT
     
      ,convert(date,[Date_of_Census]) censusdate
      
  FROM valid_census
  WHERE mrn is not null and Positive_COVID_This_Census_Date =1
  AND unit_type <>'ICU'
  Group by convert(date,Date_of_census)) covid_None_icu 
  on TotalAvailable.censusdate=covid_None_icu.censusdate
 ORDER BY TotalAvailable.censusdate DESC
'''

census_All =pd.read_sql_query(census_sql, engine)
#census_All.set_index(['censusdate'], inplace = True, append = False, drop = True)
census_All['CVD_pat_total'] =census_All.COVID_CNT

census_All.columns =['censusdate','Inpatient_max','ICU_max','Inpatient_census','ICU_census','Covid_census','CVD_ICU_pat','CVD_MedSurg_pat','CVD_pat_total']

census_All['Inpatient_census_Avg'] =census_All[census_All.censusdate >datetime.date(datetime.today() -timedelta(31))].Inpatient_census.mean()
census_All['ICU_census_Avg'] =census_All[census_All.censusdate >datetime.date(datetime.today() -timedelta(31))].ICU_census.mean()
census_All.censusdate =pd.to_datetime(census_All.censusdate)

census_copy =census_All.copy()
census_All.set_index('censusdate',inplace = True, append = False, drop = True)

census_pred =census_total_pred
census_pred['Inpatient_max'] =census_copy[census_copy.censusdate==census_All.index.max()].Inpatient_max[0]
census_pred['Inpatient_census_Avg'] =census_All.Inpatient_census_Avg[0]
census_pred['ICU_max'] =census_copy[census_copy.censusdate==census_All.index.max()].ICU_max[0]
census_pred['ICU_census_Avg'] =census_copy.ICU_census_Avg[0]
census_pred['CVD_pat_total'] =census_pred.fit
census_pred['CVD_ICU_pat'] =census_pred['CVD_pat_total'] * 0.28 ###census_ICU_pred.fit

census_pred['CVD_MedSurg_pat']=census_pred['CVD_pat_total'] -census_pred['CVD_ICU_pat']
census_pred['Inpatient_MedSurg_Total'] =census_pred['Inpatient_census_Avg'] +census_pred['CVD_MedSurg_pat']
census_pred['ICU_Total'] =census_pred['ICU_census_Avg'] + census_pred['CVD_ICU_pat']
census_pred=census_pred.reset_index(drop=False)
census_pred.set_index('Date',inplace = True, append = False, drop = True)

combined_census=census_All.append(census_pred,sort =False)
combined_census['Model_generate_date'] =date.today()

combined_census['Model_Date'] =combined_census.index
Final_census =combined_census[['Model_generate_date','Model_Date','Inpatient_census', 'ICU_census', 'Covid_census', 
       'Inpatient_max', 'Inpatient_census_Avg', 'ICU_max', 'ICU_census_Avg','CVD_pat_total', 'CVD_ICU_pat', 'CVD_MedSurg_pat',
       'Inpatient_MedSurg_Total', 'ICU_Total']]

Final_census.columns =['Model_generate_date','Model_Date','Inpatient_MedSurg_census', 'Non_CVD_ICU_census', 'CVD_census', 
       'Inpatient_MedSurg_max', 'Inpatient_MedSurg_census_Avg', 'ICU_max', 'Non_CVD_ICU_census_Avg',
       'CVD_pat_total', 'CVD_ICU_pat', 'CVD_MedSurg_pat','Inpatient_MedSurg_Total', 'ICU_Total']
#Final_census['Model_generate_date'] =datetime.today()
#Final_census['Model_Date'] =Final_census.index
#Final_census.to_csv('ICU_census.csv')
Final_census.sort_values(by='Model_Date', ascending =True)

driver='{ODBC Driver 13 for SQL Server}'
server='EDWPROD'
database='BI_Reporting'
table ='Census_Forecast'

#params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + username + ';PWD=' + password
params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';Trusted_Connection=yes'

db_params = urllib.parse.quote_plus(params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params), fast_executemany = True)
#df is the dataframe; test is table name in which this dataframe is #inserted
Final_census.to_sql(table,engine,index=False,if_exists="append",schema="dbo")